In [1]:
# import beyourself
import pandas as pd
import numpy as np
import os
from sklearn.manifold import TSNE
import time
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import confusion_matrix, mean_squared_error
from utils import balanced_subsample
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler


/Users/shibozhang/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## train: DBSACN cluster，
## test: kNN classification

In [2]:
def standard_normalizer(x):
    x = x.T
    # compute the mean and standard deviation of the input
    x_means = np.mean(x,axis = 1)[:,np.newaxis]
    x_stds = np.std(x,axis = 1)[:,np.newaxis]   

    # create standard normalizer function based on input data statistics
    normalizer = lambda data: ((data.T - x_means)/x_stds).T
    
    # return normalizer and inverse_normalizer
    return normalizer

In [3]:
def calc_multi_cm(y_gt, y_pred):    
    # ct = pd.crosstab(y_gt, y_pred, rownames=['True'], colnames=['Predicted'], margins=True).apply(lambda r: r/r.sum(), axis=1)
    ct = pd.crosstab(y_gt, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)
    # print(ct)
    # ct.to_csv(cm_file)

    # Compute confusion matrix
    multi_cm = confusion_matrix(y_gt, y_pred)
    
    accuracy = sum(multi_cm[i,i] for i in range(len(set(y_gt))))/sum(sum(multi_cm[i] for i in range(len(set(y_gt)))))
    recall_all = sum(multi_cm[i,i]/sum(multi_cm[i,j] for j in range(len(set(y_gt)))) for i in range(len(set(y_gt))))/(len(set(y_gt)))
    precision_all = sum(multi_cm[i,i]/sum(multi_cm[j,i] for j in range(len(set(y_gt)))) for i in range(len(set(y_gt))))/(len(set(y_gt)))
    fscore_all = sum(2*(multi_cm[i,i]/sum(multi_cm[i,j] for j in range(len(set(y_gt)))))*(multi_cm[i,i]/sum(multi_cm[j,i] for j in range(len(set(y_gt)))))/(multi_cm[i,i]/sum(multi_cm[i,j] for j in range(len(set(y_gt))))+multi_cm[i,i]/sum(multi_cm[j,i] for j in range(len(set(y_gt))))) for i in range(len(set(y_gt))))/len(set(y_gt))
    
    result={}

    for i in np.unique(y_gt):

        i_gt = (y_gt==i).astype(int)
        i_pred = (y_pred==i).astype(int)

        cm = confusion_matrix(i_gt, i_pred)

        i_result = {}

        TP = cm[1,1]
        FP = cm[0,1]
        TN = cm[0,0]
        FN = cm[1,0]
        # Precision for Positive = TP/(TP + FP)
        prec_pos = TP/(TP + FP)
        # F1 score for positive = 2 * precision * recall / (precision + recall)….or it can be F1= 2*TP/(2*TP + FP+ FN)
        f1_pos = 2*TP/(TP*2 + FP+ FN)
        # TPR = TP/(TP+FN)
        TPR = cm[1,1]/sum(cm[1,j] for j in range(len(set(i_gt))))

        i_result = {'recall': TPR, 'precision': prec_pos, 'f1': f1_pos}

        result[str(int(i))] = i_result

    ave_result = {'accuracy':accuracy, 'recall_all':recall_all, 'precision_all':precision_all, 'fscore_all':fscore_all}
    result['average'] = ave_result

    return result, multi_cm


In [5]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold,KFold



#read data first
fd_folder='All/fd/fd_groundtruthsegmentation.csv'
dd_folder='All/dd/dd_groundtruthsegmentation.csv'
sd_folder='All/sd/sd_groundtruthsegmentation.csv'
cd_folder='All/cd/cd_groundtruthsegmentation.csv'
null_folder='All/null/null_groundtruthsegmentation.csv'
fd,dd,sd,cd=pd.read_csv(fd_folder),pd.read_csv(dd_folder),pd.read_csv(sd_folder),pd.read_csv(cd_folder)
null=pd.read_csv(null_folder)

#definle label
positive_group=pd.concat([fd,dd])
positive_group['label']=1

negative_group=pd.concat([null])
negative_group['label']=0
total_group=pd.concat([positive_group,negative_group])

#cluster number
cluster_number=2

clf_list=[svm.SVC(),RandomForestClassifier(n_estimators=185),LogisticRegression()]#,svm.SVC(kernel='poly'),svm.SVC(kernel='sigmoid'),
clf_names=['svm(rbf)','rf','lr']#,'svm(poly)','svm(sigmoid)'

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for clf_name in clf_names:
    vars()['gt_all_folds_'+clf_name] = []
    vars()['pd_all_folds_'+clf_name] = []

iFold = 0
for XY_trn_idx, XY_test_idx in skf.split(total_group.values[:,:-1],
                                        total_group['label'].values):
    
    print('------iFold:'+str(iFold)+'---------------------')


    trn_group = total_group.iloc[XY_trn_idx]
    names = trn_group.columns
    test_group = total_group.iloc[XY_test_idx]
    X_train, X_test = trn_group.values[:,:-1], test_group.values[:,:-1]
    y_train, y_test = trn_group.values[:,-1], test_group.values[:,-1]

    # subsampling balancing will harm clustering, so cannot be used
#     X_train, y_train = balanced_subsample(X_train,y_train,subsample_size=1.0)

#     X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    X_train, y_train = ADASYN().fit_sample(X_train, y_train)
    trn_group = pd.DataFrame(data=np.hstack((X_train,y_train[:,np.newaxis])),columns=names)


    
    # normalization
    normalizer = standard_normalizer(X_train)
    X_train = normalizer(X_train)
    X_test = normalizer(X_test)
    
    #fit the data
    kmeans = MiniBatchKMeans(n_clusters=cluster_number).fit(X_train)

    # train
    # ........assign each data a label
    trn_group.loc[:,'cluster_id']=kmeans.predict(X_train)

    for i in range(cluster_number):
        vars()['XY_trn_cluster_'+str(i)] = trn_group[trn_group['cluster_id']==i]

        for (clf_type, clf_name) in zip(clf_list, clf_names):
            vars()['cluster'+str(i)+'_'+clf_name] = clf_type.fit(vars()['XY_trn_cluster_'+str(i)].values[:,:-2],
                                                                 vars()['XY_trn_cluster_'+str(i)]['label'].values)
    
    # test
    # ........assign each data a label
    test_group.loc[:,'cluster_id']=kmeans.predict(X_test)
    
    for clf_name in clf_names:
        vars()['gt_all_clusters_'+clf_name] = []
        vars()['pd_all_clusters_'+clf_name] = []
    
        for i in range(cluster_number):
            vars()['XY_test_cluster_'+str(i)] = test_group[test_group['cluster_id']==i]
            vars()['gt_all_clusters_'+clf_name].append(vars()['XY_test_cluster_'+str(i)]['label'].values)
            vars()['pd_all_clusters_'+clf_name].append(
                vars()['cluster'+str(i)+'_'+clf_name].predict(vars()['XY_test_cluster_'+str(i)].values[:,:-2]))
        
        vars()['gt_all_clusters_'+clf_name]=np.concatenate(vars()['gt_all_clusters_'+clf_name])
        vars()['pd_all_clusters_'+clf_name]=np.concatenate(vars()['pd_all_clusters_'+clf_name])

        result,_cm = calc_multi_cm(vars()['gt_all_clusters_'+clf_name], vars()['pd_all_clusters_'+clf_name])
        print('---------------------'+clf_name+'----------------')
        print(result['1']['f1'])
#         print(result)
        print(_cm)

        vars()['pd_all_folds_'+clf_name].append(vars()['pd_all_clusters_'+clf_name])
        vars()['gt_all_folds_'+clf_name].append(vars()['gt_all_clusters_'+clf_name])

    iFold += 1

print('---------------------Final----------------')

for clf_name in clf_names:
    vars()['gt_all_folds_'+clf_name]=np.concatenate(vars()['gt_all_folds_'+clf_name])
    vars()['pd_all_folds_'+clf_name]=np.concatenate(vars()['pd_all_folds_'+clf_name])
    
    result,_cm = calc_multi_cm(vars()['gt_all_folds_'+clf_name], vars()['pd_all_folds_'+clf_name])

    print('---------------------'+clf_name+'----------------')
    print(result['1']['f1'])
    print(result)
    print(_cm)


------iFold:0---------------------


/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


---------------------svm(rbf)----------------
0.11764705882352941
[[152   5]
 [ 40   3]]
---------------------rf----------------
0.4166666666666667
[[143  14]
 [ 28  15]]
---------------------lr----------------
0.4631578947368421
[[127  30]
 [ 21  22]]
------iFold:1---------------------


/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


---------------------svm(rbf)----------------
0.3466666666666667
[[ 14 143]
 [  4  39]]
---------------------rf----------------
0.673469387755102
[[135  22]
 [ 10  33]]
---------------------lr----------------
0.5648854961832062
[[106  51]
 [  6  37]]
------iFold:2---------------------


...

---------------------svm(rbf)----------------
0.3055555555555556
[[ 16 141]
 [  9  33]]
---------------------rf----------------
0.5894736842105263
[[132  25]
 [ 14  28]]
---------------------lr----------------
0.496551724137931
[[90 67]
 [ 6 36]]
------iFold:8---------------------


/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


---------------------svm(rbf)----------------
0.08695652173913043
[[154   2]
 [ 40   2]]
---------------------rf----------------
0.3448275862068966
[[150   6]
 [ 32  10]]
---------------------lr----------------
0.4666666666666667
[[129  27]
 [ 21  21]]
------iFold:9---------------------


/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shibozhang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


---------------------svm(rbf)----------------
0.34977578475336324
[[ 14 142]
 [  3  39]]
---------------------rf----------------
0.6046511627906976
[[138  18]
 [ 16  26]]
---------------------lr----------------
0.6407766990291263
[[128  28]
 [  9  33]]
---------------------Final----------------
---------------------svm(rbf)----------------
0.32529375386518244
{'0': {'recall': 0.40625, 'precision': 0.7992471769134254, 'f1': 0.5386892177589852}, '1': {'recall': 0.6217494089834515, 'precision': 0.2202680067001675, 'f1': 0.32529375386518244}, 'average': {'accuracy': 0.45203415369161226, 'recall_all': 0.5139997044917257, 'precision_all': 0.5097575918067965, 'fscore_all': 0.4319914858120838}}
[[637 931]
 [160 263]]
---------------------rf----------------
0.5443645083932853
{'0': {'recall': 0.8826530612244898, 'precision': 0.8759493670886076, 'f1': 0.8792884371029225}, '1': {'recall': 0.5366430260047281, 'precision': 0.5523114355231143, 'f1': 0.5443645083932853}, 'average': {'accuracy': 0.809